In [2]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import requests

In [3]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/train.json?TWGServiceId=kakao_arena&Expires=1625645486&Signature=WFiPQJDUs6eR/NCJ6ec3YgI%2B%2BvE%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
train=r.json()

train_df=pd.DataFrame(train)
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [4]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/genre_gn_all.json?TWGServiceId=kakao_arena&Expires=1625645502&Signature=NaZlD1FutzIbHV5Sxyu9x6qvI34%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
genre=r.json()

genre_ser=pd.Series(genre)
genre_ser

GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Length: 254, dtype: object

In [5]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/song_meta.json?TWGServiceId=kakao_arena&Expires=1625645522&Signature=eJPkX%2Bf8c1W8pSWGL4kPPu1zmtQ%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
song=r.json()

song_df=pd.DataFrame(song)
song_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [6]:
# Word2Vec 모델 로드
from gensim.models import Word2Vec

item2vec=Word2Vec.load("./item/item2vec.model2")
item2vec

In [7]:
# 학습 데이터 생성??????

import random

songs_map={s["id"]:s for s in song}

train_set=[]
for pl in train:
    songs=pl["songs"]
    tags=pl["tags"]
    gnrs=[]
    for s in songs:
        gnr=songs_map[s]["song_gn_dtl_gnr_basket"]
        gnrs.extend(gnr)
    gnrs=set(gnrs)
    if len(gnrs)<5:
        continue
    sample_size=len(songs)//7
    
    for i in range(sample_size):
        nplT={}
        
        rs=random.sample(songs,7)
        nplT["s1"]=rs[0]
        nplT["s2"]=rs[1]
        nplT["s3"]=rs[2]
        nplT["s4"]=rs[3]
        nplT["s5"]=rs[4]
        nplT["s6"]=rs[5]
        nplT["target"]=rs[6]
        nplT["label"]=1
        
        rg=random.sample(gnrs,5)
        nplT["g1"]=rg[0]
        nplT["g2"]=rg[1]
        nplT["g3"]=rg[2]
        nplT["g4"]=rg[3]
        nplT["g5"]=rg[4]
        if len(tags)>3:
            rt=random.sample(tags,3)
            nplT["t1"]=rt[0]
            nplT["t2"]=rt[1]
            nplT["t3"]=rt[2]
        elif len(tags)==0:
            nplT["t1"]=""
            nplT["t2"]=""
            nplT["t3"]=""
        elif len(tags)==1:
            nplT["t1"]=tags[0]
            nplT["t2"]=""
            nplT["t3"]=""
        elif len(tags)==2:
            nplT["t1"]=tags[0]
            nplT["t2"]=tags[1]
            nplT["t3"]=""
        train_set.append(nplT)
    while 1:
        # randint : 균일 분포의 정수 난수 1개 생성
        neg=random.randint(1,len(song))
        if neg not in songs:
            nplF=nplT.copy()
            nplF["target"]=neg
            nplF["label"]=0
            train_set.append(nplF)
            break
                
train_df=pd.DataFrame(train_set)

train_df

,s1,s2,s3,s4,s5,s6,target,label,g1,g2,g3,g4,g5,t1,t2,t3
0,297861,50104,461973,562083,660493,205238,531057,1,GN1402,GN1904,GN1102,GN1901,GN1001,락,,
1,706183,383374,351214,223955,72552,127099,324992,1,GN1005,GN1302,GN1013,GN1301,GN0904,락,,
2,706183,383374,351214,223955,72552,127099,166638,0,GN1005,GN1302,GN1013,GN1301,GN0904,락,,
3,675945,425946,41749,389529,631268,586541,37741,1,GN1608,GN0103,GN0601,GN1504,GN1501,추억,회상,
4,383960,41749,413920,117205,395416,571802,548636,1,GN0605,GN1708,GN2601,GN0509,GN1601,추억,회상,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789725,570768,627286,225448,119618,316256,310974,415749,1,GN1004,GN1307,GN1005,GN1208,GN0907,팝송,노래추천,팝송추천
789726,407370,521769,421930,693423,505710,595717,10141,1,GN1012,GN1304,GN1010,GN0901,GN1506,팝송,팝송추천,팝송모음
789727,19927,595717,255937,257800,362531,167594,229337,1,GN1304,GN1509,GN1001,GN0901,GN1301,팝송모음,팝송,팝송추천
789728,505710,89871,497011,374554,327837,676563,3179,1,GN0901,GN1013,GN1506,GN1005,GN1504,팝송추천,노래추천,팝송


In [8]:
# 학습 데이터 저장
train_df.to_csv("./keras_trainTF.csv",index=False)

In [9]:
# 학습 데이터 로드
train_df=pd.read_csv("./keras_trainTF.csv")

train_df[["t1","t2","t3"]]=train_df[["t1","t2","t3"]].fillna("-")

sg_size=len(song)

gr_nq=list(genre)

t1_nq=train_df["t1"].unique()
t2_nq=train_df["t2"].unique()
t3_nq=train_df["t3"].unique()
tg_nq=np.concatenate((t1_nq,t2_nq,t3_nq))
tg_nq=np.unique(tg_nq)

import KerasModel
# KerasModel 모델 훈련 - target 과 곡들의 유사도를 예측해 playlist에 해당 곡을 추천해줄지 결정한는 model???????
model1=KerasModel.rank_model(sg_size,tg_nq,gr_nq)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tags (InputLayer)               [(None, 3)]          0                                            
__________________________________________________________________________________________________
genres (InputLayer)             [(None, 5)]          0                                            
__________________________________________________________________________________________________
songs (InputLayer)              [(None, 6)]          0                                            
__________________________________________________________________________________________________
string_lookup (StringLookup)    (None, 3)            0           tags[0][0]                       
______________________________________________________________________________________________

In [10]:
import tensorflow as tf

# 입력 데이터 변환 
sg_train=train_df[["s1","s3","s3","s4","s5","s6"]].to_numpy()
print(sg_train)

tg_train=train_df[["t1","t2","t3"]].astype("string").to_numpy()
print(tg_train)

gr_train=train_df[["g1","g2","g3","g4","g5"]].astype("string").to_numpy()
print(gr_train)

target_train=train_df["target"].to_numpy()

y_train=train_df["label"].to_numpy()

# 콜백함수 생성
filepath="./model/rank"
mc=tf.keras.callbacks.ModelCheckpoint(
filepath,moniter="val_loss",mode="auto",save_best_only=True,save_weights_only=True)

# 모델 학습
model.fit({"songs":sg_train,"tags":tg_train,"genres":gr_train,"target":target_train},y_train,batch_size=256,
         epochs=3,
         callbacks=[mc],
         validation_split=0.2)

[[297861 461973 461973 562083 660493 205238]
 [706183 351214 351214 223955  72552 127099]
 [706183 351214 351214 223955  72552 127099]
 ...
 [ 19927 255937 255937 257800 362531 167594]
 [505710 497011 497011 374554 327837 676563]
 [505710 497011 497011 374554 327837 676563]]
[['락' '-' '-']
 ['락' '-' '-']
 ['락' '-' '-']
 ...
 ['팝송모음' '팝송' '팝송추천']
 ['팝송추천' '노래추천' '팝송']
 ['팝송추천' '노래추천' '팝송']]
[['GN1402' 'GN1904' 'GN1102' 'GN1901' 'GN1001']
 ['GN1005' 'GN1302' 'GN1013' 'GN1301' 'GN0904']
 ['GN1005' 'GN1302' 'GN1013' 'GN1301' 'GN0904']
 ...
 ['GN1304' 'GN1509' 'GN1001' 'GN0901' 'GN1301']
 ['GN0901' 'GN1013' 'GN1506' 'GN1005' 'GN1504']
 ['GN0901' 'GN1013' 'GN1506' 'GN1005' 'GN1504']]
Epoch 1/3
2468/2468 [==============================] - 6182s 3s/step - loss: 0.3568 - val_loss: 0.2804
Epoch 2/3
2468/2468 [==============================] - 6335s 3s/step - loss: 0.1302 - val_loss: 0.3544
Epoch 3/3
2468/2468 [==============================] - 6320s 3s/step - loss: 0.0477 - val_loss: 0.4502


In [11]:
# 모델 저장
model.save_weights("./model/rankover")

In [ ]:
import random

model.load_weights("./model/search")
model1.load_weights("./model/rank")

songs_map={s["id"]: s for s in song}

for pl in train[:30]:
    songs=pl["songs"]
    tags=pl["tags"]
    gnrs=[]
    for s in songs:
        gnr=songs_map[s]["song_gn_dtl_gnr_basket"]
        gnrs.extend(gnr)
        
    gnrs=set(gnrs)
    
    if len(gnrs)<5 or len(songs)<7:
        continue
        
    sg_test=random.sample(songs,6)
    gr_test=random.sample(gnrs,5)
    tg_test=["-","-","-"]
    if len(tags)>3:
        tg_test=random.sample(tags,3)
    else:
        for i,t in enumerate(tags):
            tf_test[i]=t
    tg_test=np.array(tg_test)
    
    # 
    psongs=model1.predict({"songs":np.array([sg_test]),"tags":np.array([tg_test]),"genres":np.array([gr_test])})
    
    candidate=np.argsort(psongs[0])[-10:]
    print(psongs[0][candidate])
    
    socre=np.zeros((10,))
    for i,cs in enumerate(candidate):
        psongs=model2.predict({"songs":np.array([sg_test]),"tags":np.array([tg_test]),
                               "genres":np.array([gr_test]),"target":np.array([cs])})
        score[i]=psongs[0]
    
    for sg in song_test:
        for i,cs in enumerate(candidate):
            
            psongs=model2.predict(genre)
            score[i]=psongs[0]

        
    top5=np.argsort(score)[-5:]
    print(score[top5])
    print(candidate[top5])
    acc=[p for p in candidate[top3] if p in songs]
    print(acc)

In [ ]:
#?????????????????